In [8]:
# !pip install transformers
# !pip install torch

In [27]:
collected_data = {
    "room": "거실 / 거실의 빈 벽이 너무 심심해서 놓고 싶어",
    "humidity": "건조",
    "preferred_style": "미니멀",
    "preferred_color": "따뜻한 색",
    "has_dog": True,
    "has_cat": False,
    "isAirCond": True,
    "watering_frequency": "주 1회",
    "user_experience": "쉬움"
}

In [28]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_path = "Dongjin-kr/ko-reranker"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def rerank(query, documents):
    texts = [
        doc.page_content if hasattr(doc, "page_content") 
        else doc.get("text") if isinstance(doc, dict) 
        else doc
        for doc in documents
    ]

    pairs = [[query, text] for text in texts]
    
    inputs = tokenizer(
        pairs, 
        padding=True, 
        truncation=True, 
        return_tensors='pt'
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        scores = outputs.logits.squeeze(-1)
    
    ranked = sorted(
        list(zip(documents, scores.cpu().tolist())),
        key=lambda x: x[1],
        reverse=True
    )
    
    return ranked


In [16]:
collected_data['has_cat']

False

In [29]:
from config import REC_INDEX_NAME, pc, embeddings
from langchain_pinecone import PineconeVectorStore

def tool_rag_recommend(query) -> str:
    index = pc.Index(REC_INDEX_NAME)

    vector_store = PineconeVectorStore(
        index=index,
        embedding=embeddings
    )

    filter_conditions = {}

    # 강아지 있을 경우, 독성 False만
    if query.get("has_dog") == True:
        filter_conditions["isToxicToDog"] = {"$eq": False}

    # 고양이 있을 경우, 독성 False만
    if query.get("has_cat") == True:
        filter_conditions["isToxicToCat"] = {"$eq": False}

    retriever = vector_store.as_retriever(
        search_kwargs={
            "k": 5,
            "filter": filter_conditions
        }
    )   

    retrievals = retriever.batch([str(query)])[0]

    if not retrievals:
        return str({"error": "검색 결과 없음", "filter_used": filter_conditions})

    ranked = rerank(str(query), retrievals)

    best_doc, best_score = ranked[0]

    return str({
        "best": best_doc.page_content if hasattr(best_doc, "page_content") else str(best_doc),
        "score": best_score,
        "all_ranked": [
            (d.page_content if hasattr(d, "page_content") else str(d), s) 
            for d, s in ranked
        ]
    })


In [30]:
res = tool_rag_recommend(collected_data)

In [31]:
print(res)

{'best': "인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다., 돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.. 공기정화식물이다. 강아지에게 안전하다. 고양이에게 안전하다. #D43737 #333F25 #DE915A #4A3007 #3B3025 #B47C69", 'score': -6.496788024902344, 'all_ranked': [("인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서

---

In [ ]:
# reranking 전
def recommend(query: str) -> str:
    """식물 추천 전용 RAG 도구"""
    index = pc.Index(REC_INDEX_NAME)

    vector_store = PineconeVectorStore(
        index=index,
        embedding=embeddings
    )
    
    retriever = vector_store.as_retriever(search_kwargs={"k": 1})
    retrievals = retriever.batch([query])
    
    return str(retrievals[0])

In [22]:
response = recommend(collected_data)

In [27]:
print(response)

[Document(id='132', metadata={'dataNo': '132', 'difficulty': '쉬움', 'fContent': '하늘에 흘러가는 구름을 닮고픈 걸까, 이글거리는 태양을 닮고픈 걸까. 여러 자생 나리들 중에서 진짜 나리는 나야! 라고 소리치는 듯 하다. 당당하게 하늘을 우러러 보는 모습이 때 묻지 않은 산골소녀의 해맑은 모습을 보는 것 같기도 하다.', 'fDay': '11', 'fGrow': '여름철 고온다습한 환경에 약하므로 배수가 잘되는 곳에 심는다. 가장 널리 이용하는 방법이 인편번식으로 깨끗한 삽목상에 심는다. 종자번식도 잘되는데 씨앗을 뿌리면 3년은 지나야 꽃이 핀다. 나리류 중에서 참나리만 주아(꽃대의 잎겨드랑이에 검은색 열매처럼 보이는)가 달리는데 이 주아를 뿌린다.', 'fMonth': '5', 'fType': '백합과의 여러해살이 풀이다. 깊은 산 햇빛이 잘 드는 곳에서 자라며 긴 선형의 잎이 어긋나게 달린다. 짙은 주홍빛에 자주색반점이 있는 꽃이 하늘을 향해 핀다.', 'fUse': '꽃이 화려한데다 크고 색상이 강렬하여 눈에 잘 뜨인다. 정원의 경계부나 화단의 포인트가 될 부분에 모아심기 해 두면 참 잘 어울린다. 가정에 심어두고 한창 개화기 때 꽃을 잘라 꽃꽂이용으로 이용해도 좋다.', 'flowLang': '길들여지지 않음, 변치 않는 귀여움', 'flowNm': '하늘나리', 'imgUrl1': 'https://www.nihhs.go.kr/Attfiles/FLOWF/0511-3.jpg', 'imgUrl2': 'https://www.nihhs.go.kr/Attfiles/FLOWF/0511-2.jpg', 'imgUrl3': 'https://www.nihhs.go.kr/Attfiles/FLOWF/0511-1.jpg', 'interior': '하늘나리는 밝은 색상의 화분에 심어 자연스러운 느낌을 주며, 곁에 다른 식물들과 함께 배치하여 생동감 있는 공간을 연출할 수 있습니다.', 'isAirCond': False

---

In [ ]:
# reranking 전 k=1
page_content='길들여지지 않음, 변치 않는 귀여움, 하늘에 흘러가는 구름을 닮고픈 걸까, 이글거리는 태양을 닮고픈 걸까. 여러 자생 나리들 중에서 진짜 나리는 나야! 라고 소리치는 듯 하다. 당당하게 하늘을 우러러 보는 모습이 때 묻지 않은 산골소녀의 해맑은 모습을 보는 것 같기도 하다., 물주기: 주 1회, 난이도 쉬움, 하늘나리는 밝은 색상의 화분에 심어 자연스러운 느낌을 주며, 곁에 다른 식물들과 함께 배치하여 생동감 있는 공간을 연출할 수 있습니다., 모던, 스칸디나비아, 보헤미안, 꽃이 화려한데다 크고 색상이 강렬하여 눈에 잘 뜨인다. 정원의 경계부나 화단의 포인트가 될 부분에 모아심기 해 두면 참 잘 어울린다. 가정에 심어두고 한창 개화기 때 꽃을 잘라 꽃꽂이용으로 이용해도 좋다., 여름철 고온다습한 환경에 약하므로 배수가 잘되는 곳에 심는다. 가장 널리 이용하는 방법이 인편번식으로 깨끗한 삽목상에 심는다. 종자번식도 잘되는데 씨앗을 뿌리면 3년은 지나야 꽃이 핀다. 나리류 중에서 참나리만 주아(꽃대의 잎겨드랑이에 검은색 열매처럼 보이는)가 달리는데 이 주아를 뿌린다., 백합과의 여러해살이 풀이다. 깊은 산 햇빛이 잘 드는 곳에서 자라며 긴 선형의 잎이 어긋나게 달린다. 짙은 주홍빛에 자주색반점이 있는 꽃이 하늘을 향해 핀다..  강아지에게 안전하다. 고양이에게 안전하다. #283B27 #667D3F #474913 #918B42 #6A8136 #E95124'

In [ ]:
# reranking k=10
{'best': '행운, 공기정화식물도 잘 알려져 있다. 손가락같이 갈라진 잎과 뿌리 윗부분의 줄기가 통통한 모양새가 특이한 이국적인 식물이다. 자생지인 멕시코에서는 교목성으로 자라지만 우리나라에서는 화분에 심어 잎을 관상하는 관엽식물로 자리잡고 있다., 물주기: 주 1회, 난이도 쉬움, 파키라는 실내에서 높고 넓은 공간을 채우기 좋으며, 화이트나 연한 색상의 화분에 심어 포인트를 줄 수 있습니다. 또한, 잎이 풍성해 공간에 생동감을 더해줍니다., 모던, 스칸디나비안, 보헤미안, 잎은 시원하고 줄기는 매끈하며 특히 줄기 아랫부분은 항아리모양으로 불룩하게 비대되어 모양이 특이하며 보기가 좋아 집안의 거실이나 건물의 실내식물로 잘 어울린다. 현재 이용되는 것은 대형종이 많지만 키가 작은 소형종도 있다., 전형적인 남방계식물로 추위에 약해 겨울에도 최저 10℃ 이상은 되어야 별탈없이 월동할 수 있다. 보통은 16-30℃정도가 알맞다. 실내라 하더라도 햇볕이 잘 드는 곳이 좋으며 환기만 잘 된다면 습도가 높을수록 잘 자란다., 물밤나무과에 속하며 원산지인 남미에서는 열매를 식용으로 쓰기도 한다..  강아지에게 위험하다. 고양이에게 안전하다. #A2C492 #3C542E #1C2F13 #537550 #364A31 #779479', 'score': -6.508496284484863, 
 'all_ranked': [
    ('행운, 공기정화식물도 잘 알려져 있다. 손가락같이 갈라진 잎과 뿌리 윗부분의 줄기가 통통한 모양새가 특이한 이국적인 식물이다. 자생지인 멕시코에서는 교목성으로 자라지만 우리나라에서는 화분에 심어 잎을 관상하는 관엽식물로 자리잡고 있다., 물주기: 주 1회, 난이도 쉬움, 파키라는 실내에서 높고 넓은 공간을 채우기 좋으며, 화이트나 연한 색상의 화분에 심어 포인트를 줄 수 있습니다. 또한, 잎이 풍성해 공간에 생동감을 더해줍니다., 모던, 스칸디나비안, 보헤미안, 잎은 시원하고 줄기는 매끈하며 특히 줄기 아랫부분은 항아리모양으로 불룩하게 비대되어 모양이 특이하며 보기가 좋아 집안의 거실이나 건물의 실내식물로 잘 어울린다. 현재 이용되는 것은 대형종이 많지만 키가 작은 소형종도 있다., 전형적인 남방계식물로 추위에 약해 겨울에도 최저 10℃ 이상은 되어야 별탈없이 월동할 수 있다. 보통은 16-30℃정도가 알맞다. 실내라 하더라도 햇볕이 잘 드는 곳이 좋으며 환기만 잘 된다면 습도가 높을수록 잘 자란다., 물밤나무과에 속하며 원산지인 남미에서는 열매를 식용으로 쓰기도 한다..  강아지에게 위험하다. 고양이에게 안전하다. #A2C492 #3C542E #1C2F13 #537550 #364A31 #779479', -6.508496284484863), 
    ('행운, 행복, 생활공간에서 이용하는 관엽류 중 가장 많이 쓰는 게 바로 행운목이 아닐까 싶다. 식물의 이름이나 꽃말에서처럼 행운, 행복, lucky, happiness 등의 용어는 동서양을 막론하고 참 좋아하는 단어인 것 같다., 물주기: 주 1회, 난이도 쉬움, 행운목은 테이블이나 책상에 두어 생활 공간에 생기를 더할 수 있으며, 조명 아래 배치하면 더욱 아늑한 분위기를 제공합니다., 모던, 스칸디나비아, 미니멀리즘, 화분에 심어 실내에서 관상용으로 기르는 것이 일반적이다. 건물의 사무실이나 학교, 가정의 베란다에 실내정원을 만들 때 중요한 중심소재로 쓰기도 한다. 종에 따라 키가 6m까지 자라는 것도 있으며 잎이 가늘고 길어 시원한 느낌을 준다., 행운목은 드라세나류 중의 하나이다. 여름에 햇볕이 강하면 잎이 타는 경우가 있어 반그늘 정도의 차광이 필요하다. 대신 겨울에는 볕이 잘 드는 곳에 두어야 고유의 잎 색을 낸다. 생장에 알맞은 온도는 20-25℃이며, 추위에도 강한 편으로 겨울에 12℃ 정도에도 잘 견딘다. 하지만 잎에 무늬가 있는 종들은 추위에 다소 약한 편으로 15℃ 이상은 되어야 정상생육이 가능하다., 아스파라거스과에 속하는 상록성의 식물로 약 40종이 카나리 반도와 아프리카 서부나 이디오피아 열대지역에 자생한다.. 공기정화식물이다. 강아지에게 위험하다. 고양이에게 안전하다. #7BB355 #2E6613 #233807 #7EA53E #24340E #72884C', -7.113885879516602), 
    ('흉내, 감국과 비슷하게 피면서 감국인 것처럼 흉내를 내지만 인기가 적어 사람들의 관심을 끌지 못한다. 꽃도 작아 볼품 적지만 향기도 너무 강해 차로도 마실 수가 없기 때문이다., 물주기: 주 1회, 난이도 쉬움, 산국은 그린 인테리어에 적합하며, 화분에 심어 창가에 두거나, 책상 위에 두어 자연을 느낄 수 있도록 연출할 수 있습니다. 또한, 다른 식물과 함께 그룹으로 배치하면 조화롭고 아늑한 공간을 만들 수 있습니다., 미니멀리즘, 내추럴, 보헤미안, 감국보다 꽃은 작지만 꽃이 더 많이 달기고 늦게까지 피기 때문에 화단용으로 좋다. 가을철 도로변에 심어도 좋으며 웬만큼 건조하고나 척박해도 잘 자라 화단용으로 폭넓게 쓸 수 있다., 추위나 더위는 물론 건조에도 잘 견뎌 전국 어디서든 기를 수 있다. 씨앗으로 번식이 잘 된다. 이른 봄에 기부에서 나오는 측아를 5-10cm길이로 잘라 아래 잎 2-3매를 떼어낸 뒤 모래에 꽂으면 쉽게 새뿌리를 내린다., 국화과의 여러살이풀이다. 러시아, 중국, 우리나라, 일본에 널리 분포한다..  강아지에게 위험하다. 고양이에게 안전하다. #1F310A #D6C003 #C0BA0E #2B4219 #4F6F96 #656415', -8.210665702819824), 
    ('길들여지지 않음, 변치 않는 귀여움, 하늘에 흘러가는 구름을 닮고픈 걸까, 이글거리는 태양을 닮고픈 걸까. 여러 자생 나리들 중에서 진짜 나리는 나야! 라고 소리치는 듯 하다. 당당하게 하늘을 우러러 보는 모습이 때 묻지 않은 산골소녀의 해맑은 모습을 보는 것 같기도 하다., 물주기: 주 1회, 난이도 쉬움, 하늘나리는 밝은 색상의 화분에 심어 자연스러운 느낌을 주며, 곁에 다른 식물들과 함께 배치하여 생동감 있는 공간을 연출할 수 있습니다., 모던, 스칸디나비아, 보헤미안, 꽃이 화려한데다 크고 색상이 강렬하여 눈에 잘 뜨인다. 정원의 경계부나 화단의 포인트가 될 부분에 모아심기 해 두면 참 잘 어울린다. 가정에 심어두고 한창 개화기 때 꽃을 잘라 꽃꽂이용으로 이용해도 좋다., 여름철 고온다습한 환경에 약하므로 배수가 잘되는 곳에 심는다. 가장 널리 이용하는 방법이 인편번식으로 깨끗한 삽목상에 심는다. 종자번식도 잘되는데 씨앗을 뿌리면 3년은 지나야 꽃이 핀다. 나리류 중에서 참나리만 주아(꽃대의 잎겨드랑이에 검은색 열매처럼 보이는)가 달리는데 이 주아를 뿌린다., 백합과의 여러해살이 풀이다. 깊은 산 햇빛이 잘 드는 곳에서 자라며 긴 선형의 잎이 어긋나게 달린다. 짙은 주홍빛에 자주색반점이 있는 꽃이 하늘을 향해 핀다..  강아지에게 안전하다. 고양이에게 안전하다. #283B27 #667D3F #474913 #918B42 #6A8136 #E95124', -8.279547691345215), 
    ('하얀 미소, 바닷가나 산자락 아래 평지에서 여름이면 바닥에 하얗게 깔리면서 피는 꽃이다. 찔레랑 형제간 사이라 꽃 모양이 비슷해 보이지만 자세히 보면 많이 다르다., 물주기: 1주일에 1회, 난이도 쉬움, 돌가시나무는 밝은 곳에 두거나 간접광이 드는 자리에 배치하면 좋습니다. 소형 화분에 심어 테이블이나 창가에 두거나, 여러 개의 화분을 군집 배치하여 자연스러운 느낌을 연출할 수 있습니다., 스칸디나비안, 미니멀리즘, 보헤미안, 전남이나 경남의 남부 해안가에 주로 자생한다. 겨울이 춥지 않은 남부지역에서는 정원이나 울타리용으로 이용할 수 있다. 꽃에 향기가 나며 나중에 열매가 빨갛고 둥글게 달린다., 장미과답게 몸 전체에 가시가 많다. 정원이나 자연학습장 같은 생태공원의 바위틈, 생울타리, 경계부분 등에 심으면 잘 어울린다. 햇볕이 충분히 들고 거름기가 좀 있는 곳을 좋아한다., 반상록성의 덩굴성으로 장미과에 속한다. 우리나라, 일본, 중국 남부, 대만 등 아열대 지역에 자생한다..  강아지에게 안전하다. 고양이에게 안전하다. #3E4A1D #9EA170 #A8A082 #5A6031 #B2B297 #586D31', -9.571039199829102)]}


In [ ]:
{'best': "인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다., 돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.. 공기정화식물이다. 강아지에게 위험하다. 고양이에게 위험하다. #D43737 #333F25 #DE915A #4A3007 #3B3025 #B47C69", 'score': -6.438308238983154, 
 'all_ranked': [
   ("인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다., 돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.. 공기정화식물이다. 강아지에게 위험하다. 고양이에게 위험하다. #D43737 #333F25 #DE915A #4A3007 #3B3025 #B47C69", -6.438308238983154),         
   ('행운, 공기정화식물도 잘 알려져 있다. 손가락같이 갈라진 잎과 뿌리 윗부분의 줄기가 통통한 모양새가 특이한 이국적인 식물이다. 자생지인 멕시코에서는 교목성으로 자라지만 우리나라에서는 화분에 심어 잎을 관상하는 관엽식물로 자리잡고 있다., 물주기: 주 1회, 난이도 쉬움, 파키라는 실내에서 높고 넓은 공간을 채우기 좋으며, 화이트나 연한 색상의 화분에 심어 포인트를 줄 수 있습니다. 또한, 잎이 풍성해 공간에 생동감을 더해줍니다., 모던, 스칸디나비안, 보헤미안, 잎은 시원하고 줄기는 매끈하며 특히 줄기 아랫부분은 항아리모양으로 불룩하게 비대되어 모양이 특이하며 보기가 좋아 집안의 거실이나 건물의 실내식물로 잘 어울린다. 현재 이용되는 것은 대형종이 많지만 키가 작은 소형종도 있다., 전형적인 남방계식물로 추위에 약해 겨울에도 최저 10℃ 이상은 되어야 별탈없이 월동할 수 있다. 보통은 16-30℃정도가 알맞다. 실내라 하더라도 햇볕이 잘 드는 곳이 좋으며 환기만 잘 된다면 습도가 높을수록 잘 자란다., 물밤나무과에 속하며 원산지인 남미에서는 열매를 식용으로 쓰기도 한다..  강아지에게 안전하다. 고양이에게 위험하다. #A2C492 #3C542E #1C2F13 #537550 #364A31 #779479', -7.083730220794678), 
   ('안심하세요, 이름만 들으면 볼품없을 것 같지만 이른 봄 햇빛이 좀 드는 숲 따뜻한 곳에 무리지어 있는 모습을 보면 누구나 감탄할 정도로 아름답다. 키는 작지만 올망졸망 모여서 나고 가늘고 긴 꽃대에 보라색 꽃을 피우며 연잎처럼 생긴 잎을 가지고 있다., 물주기: 1주일에 1회, 난이도 쉬움, 깽깽이풀은 밝은 곳에서 잘 자라며, 화이트 또는 우드톤의 선반이나 테이블 위에 놓아두면 공간을 생동감 있게 연출할 수 있습니다. 또한, 식물의 고유의 느낌을 살리기 위해 같은 스타일의 화분을 이용하여 여러 개를 줄지어 배치하면 더욱 멋진 인테리어를 완성할 수 있습니다., 스칸디나비안, 내추럴, 미니멀리즘, 꽃은 물론 잎 모양도 뛰어나 화단은 물론 분화용으로도 훌륭한 소재다. 특히 추위에 강해 거의 전국에서 월동한다. 자생지에서는 무척 귀하게 여겨져 환경부 지정 보호야생식물 27호이다. 뿌리는 약용으로 쓰이기도 한다., 화단용은 바람이 잘 통하면서 반 그늘진 곳에 심는다. 분화용은 여름 고온기에 약 50%정도 차광이 필요하다. 포기나누기나 씨앗으로 번식하는데 씨앗은 뿌린 후 3년은 지나야 꽃을 볼 수 있다. 씨앗번식과 관련해서 재미난 것은 깽깽이풀 씨앗에 밀선이 있어서 여기에 들어있는 당분을 개미들이 영양원으로 사용한다. 개미는 생존을 위해 당분을 이용하고 깽깽이풀 역시 종피에 당이 벗겨지면서 쉽게 발아하게 되고, 또 개미가 이동한 만큼 자손을 멀리 퍼트릴 수 있게 되어 서로 공생관계에 있다., 매자나무과 여러해살이풀로써 지구상에 딱 2종이 있는데, 그 중 하나가 우리나라에 나는 것이다. 제주와 남부도서지방을 제외한 전국의 숲 가장자리 그늘진 곳에 자생한다..  강아지에게 위험하다. 고양이에게 위험하다. #4D3928 #978A7D #B7A3D6 #3D2321 #A193C1 #403834', -7.186977863311768), 
   ('강인함, 소나무처럼 잎이 치밀한 침엽수 아래서는 다른 식물이 잘 자랄 수 없다. 잎이 워낙 조밀 조밀한 데다 여기서 나오는 발아억제물질 같은 화학성분들이 다른 식물들을 자라지 못하게 만들기 때문이다. 이런 곳에서 산거울은 자생하고 있으니 강인함이 대단한 식물이다., 물주기: 주 1회, 난이도 쉬움, 산거울은 간결한 디자인의 화분에 심어 책상 한쪽에 배치하면, 공간에 자연스러운 느낌을 더해줍니다. 조명이 잘 드는 곳에 두면 더욱 건강하게 자라며 인테리어 효과를 높일 수 있습니다., 미니멀리즘, 내추럴, 스칸디나비안, 산거울은 소나무 같이 바늘잎을 가진 침엽수아래에 식재할 수 있는 사초류 중의 하나이다. 한 겨울을 지난 잎은 갈색으로 길게 늘어지며 이른 봄 짧은 꽃대위에 이삭모양으로 꽃을 피워낸다. 반그늘 이상의 음지 조경용으로 적합하다., 일반 사초과 식물들과 마찬가지로 씨앗번식이 잘 된다. 생육이 왕성해 포기나누기를 해도 빨리 증식할 수 있다., 사초과에 속하는 여러해살이풀이다. 그늘진 바위틈이나 건조한 숲속에 자생한다..  강아지에게 위험하다. 고양이에게 위험하다. #42432C #ABADA0 #45402C #AEA395 #533E21 #B0956B', -7.272218227386475), 
   ('길들여지지 않음, 변치 않는 귀여움, 하늘에 흘러가는 구름을 닮고픈 걸까, 이글거리는 태양을 닮고픈 걸까. 여러 자생 나리들 중에서 진짜 나리는 나야! 라고 소리치는 듯 하다. 당당하게 하늘을 우러러 보는 모습이 때 묻지 않은 산골소녀의 해맑은 모습을 보는 것 같기도 하다., 물주기: 주 1회, 난이도 쉬움, 하늘나리는 밝은 색상의 화분에 심어 자연스러운 느낌을 주며, 곁에 다른 식물들과 함께 배치하여 생동감 있는 공간을 연출할 수 있습니다., 모던, 스칸디나비아, 보헤미안, 꽃이 화려한데다 크고 색상이 강렬하여 눈에 잘 뜨인다. 정원의 경계부나 화단의 포인트가 될 부분에 모아심기 해 두면 참 잘 어울린다. 가정에 심어두고 한창 개화기 때 꽃을 잘라 꽃꽂이용으로 이용해도 좋다., 여름철 고온다습한 환경에 약하므로 배수가 잘되는 곳에 심는다. 가장 널리 이용하는 방법이 인편번식으로 깨끗한 삽목상에 심는다. 종자번식도 잘되는데 씨앗을 뿌리면 3년은 지나야 꽃이 핀다. 나리류 중에서 참나리만 주아(꽃대의 잎겨드랑이에 검은색 열매처럼 보이는)가 달리는데 이 주아를 뿌린다., 백합과의 여러해살이 풀이다. 깊은 산 햇빛이 잘 드는 곳에서 자라며 긴 선형의 잎이 어긋나게 달린다. 짙은 주홍빛에 자주색반점이 있는 꽃이 하늘을 향해 핀다..  강아지에게 위험하다. 고양이에게 위험하다. #283B27 #667D3F #474913 #918B42 #6A8136 #E95124', -7.880829334259033)]}


In [ ]:
# 필터링 & reranking k=5
{'best': "인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다., 돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.. 공기정화식물이다. 강아지에게 안전하다. 고양이에게 안전하다. #D43737 #333F25 #DE915A #4A3007 #3B3025 #B47C69", 'score': -6.496788024902344, 'all_ranked': [("인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다., 돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.. 공기정화식물이다. 강아지에게 안전하다. 고양이에게 안전하다. #D43737 #333F25 #DE915A #4A3007 #3B3025 #B47C69", -6.496788024902344), ('안심하세요, 이름만 들으면 볼품없을 것 같지만 이른 봄 햇빛이 좀 드는 숲 따뜻한 곳에 무리지어 있는 모습을 보면 누구나 감탄할 정도로 아름답다. 키는 작지만 올망졸망 모여서 나고 가늘고 긴 꽃대에 보라색 꽃을 피우며 연잎처럼 생긴 잎을 가지고 있다., 물주기: 1주일에 1회, 난이도 쉬움, 깽깽이풀은 밝은 곳에서 잘 자라며, 화이트 또는 우드톤의 선반이나 테이블 위에 놓아두면 공간을 생동감 있게 연출할 수 있습니다. 또한, 식물의 고유의 느낌을 살리기 위해 같은 스타일의 화분을 이용하여 여러 개를 줄지어 배치하면 더욱 멋진 인테리어를 완성할 수 있습니다., 스칸디나비안, 내추럴, 미니멀리즘, 꽃은 물론 잎 모양도 뛰어나 화단은 물론 분화용으로도 훌륭한 소재다. 특히 추위에 강해 거의 전국에서 월동한다. 자생지에서는 무척 귀하게 여겨져 환경부 지정 보호야생식물 27호이다. 뿌리는 약용으로 쓰이기도 한다., 화단용은 바람이 잘 통하면서 반 그늘진 곳에 심는다. 분화용은 여름 고온기에 약 50%정도 차광이 필요하다. 포기나누기나 씨앗으로 번식하는데 씨앗은 뿌린 후 3년은 지나야 꽃을 볼 수 있다. 씨앗번식과 관련해서 재미난 것은 깽깽이풀 씨앗에 밀선이 있어서 여기에 들어있는 당분을 개미들이 영양원으로 사용한다. 개미는 생존을 위해 당분을 이용하고 깽깽이풀 역시 종피에 당이 벗겨지면서 쉽게 발아하게 되고, 또 개미가 이동한 만큼 자손을 멀리 퍼트릴 수 있게 되어 서로 공생관계에 있다., 매자나무과 여러해살이풀로써 지구상에 딱 2종이 있는데, 그 중 하나가 우리나라에 나는 것이다. 제주와 남부도서지방을 제외한 전국의 숲 가장자리 그늘진 곳에 자생한다..  강아지에게 안전하다. 고양이에게 안전하다. #4D3928 #978A7D #B7A3D6 #3D2321 #A193C1 #403834', -7.186977863311768), ('환희, 열정, 이름만큼이나 색상이 곱고 꽃말처럼 모양이 열정적이다. 흰색이 많으며 노란색, 분홍색 등의 유색계도 흔히 나온다. 꽃색이 선명하며 모양이 매우 아름답다. 관상부위인 꽃은 실제로는 잎이 변형된 것으로 화포(spathe)이며, 꽃 조직인 암수술은 화포에 싸여 작은 이삭모양으로 안에 달려있다., 물주기: 주 1회, 난이도 쉬움, 칼라는 화이트 톤의 인테리어와 잘 어울리므로 밝은 배경의 창가에 배치하여 자연광을 최대한 활용합니다. 또한, 식물 독특한 잎 모양이 시각적인 포인트가 되어 인테리어의 중심이 될 수 있습니다., 미니멀리즘, 보태니컬, 북유럽, 여름에 시원하고 겨울이 따뜻한 곳에서는 화단용으로 기를 수 있다. 우리나라에서는 개화기간이 길어 분화용으로도 쓰이지만 꽃꽂이용으로 많이 쓰인다. 결혼식 신부의 부케용 또는 각종 고급 꽃바구니나 꽃다발 만들 때 주 재료로 쓰인다., 칼라는 봄에 심는 알뿌리식물로써 줄기뿌리와 알뿌리인 것이 있다. 습지형(흰색계)과 일반형(유색계)으로도 구분된다. 습지형은 봄에 포기나누기 또는 씨앗으로 번식이 잘 된다. 그늘진 곳이면 꽃이 잘 피지 않고 잎이 잘 자란다. 남부의 따뜻한 곳에서는 봄에 정원의 나무 그늘 밑에 심어 주로 잎 관상용으로 길러볼만 하다., 천남성과에 속하는 알뿌리식물이다. 아프리카 남부와 동부의 습지, 늪지 호수 가장자리 등이 고향으로 자생지에 약 6종이 분포한다.. 공기정화식물이다. 강아지에게 안전하다. 고양이에게 안전하다. #DAD7D4 #34281A #234011 #C19B65 #3E390E #A4884E', -7.2399115562438965), ('강인함, 소나무처럼 잎이 치밀한 침엽수 아래서는 다른 식물이 잘 자랄 수 없다. 잎이 워낙 조밀 조밀한 데다 여기서 나오는 발아억제물질 같은 화학성분들이 다른 식물들을 자라지 못하게 만들기 때문이다. 이런 곳에서 산거울은 자생하고 있으니 강인함이 대단한 식물이다., 물주기: 주 1회, 난이도 쉬움, 산거울은 간결한 디자인의 화분에 심어 책상 한쪽에 배치하면, 공간에 자연스러운 느낌을 더해줍니다. 조명이 잘 드는 곳에 두면 더욱 건강하게 자라며 인테리어 효과를 높일 수 있습니다., 미니멀리즘, 내추럴, 스칸디나비안, 산거울은 소나무 같이 바늘잎을 가진 침엽수아래에 식재할 수 있는 사초류 중의 하나이다. 한 겨울을 지난 잎은 갈색으로 길게 늘어지며 이른 봄 짧은 꽃대위에 이삭모양으로 꽃을 피워낸다. 반그늘 이상의 음지 조경용으로 적합하다., 일반 사초과 식물들과 마찬가지로 씨앗번식이 잘 된다. 생육이 왕성해 포기나누기를 해도 빨리 증식할 수 있다., 사초과에 속하는 여러해살이풀이다. 그늘진 바위틈이나 건조한 숲속에 자생한다..  강아지에게 안전하다. 고양이에게 안전하다. #42432C #ABADA0 #45402C #AEA395 #533E21 #B0956B', -7.260007381439209), ('길들여지지 않음, 변치 않는 귀여움, 하늘에 흘러가는 구름을 닮고픈 걸까, 이글거리는 태양을 닮고픈 걸까. 여러 자생 나리들 중에서 진짜 나리는 나야! 라고 소리치는 듯 하다. 당당하게 하늘을 우러러 보는 모습이 때 묻지 않은 산골소녀의 해맑은 모습을 보는 것 같기도 하다., 물주기: 주 1회, 난이도 쉬움, 하늘나리는 밝은 색상의 화분에 심어 자연스러운 느낌을 주며, 곁에 다른 식물들과 함께 배치하여 생동감 있는 공간을 연출할 수 있습니다., 모던, 스칸디나비아, 보헤미안, 꽃이 화려한데다 크고 색상이 강렬하여 눈에 잘 뜨인다. 정원의 경계부나 화단의 포인트가 될 부분에 모아심기 해 두면 참 잘 어울린다. 가정에 심어두고 한창 개화기 때 꽃을 잘라 꽃꽂이용으로 이용해도 좋다., 여름철 고온다습한 환경에 약하므로 배수가 잘되는 곳에 심는다. 가장 널리 이용하는 방법이 인편번식으로 깨끗한 삽목상에 심는다. 종자번식도 잘되는데 씨앗을 뿌리면 3년은 지나야 꽃이 핀다. 나리류 중에서 참나리만 주아(꽃대의 잎겨드랑이에 검은색 열매처럼 보이는)가 달리는데 이 주아를 뿌린다., 백합과의 여러해살이 풀이다. 깊은 산 햇빛이 잘 드는 곳에서 자라며 긴 선형의 잎이 어긋나게 달린다. 짙은 주홍빛에 자주색반점이 있는 꽃이 하늘을 향해 핀다..  강아지에게 안전하다. 고양이에게 안전하다. #283B27 #667D3F #474913 #918B42 #6A8136 #E95124', -7.880829334259033)]}


In [ ]:
# reranking k=5
{'best': '기쁨의 연속, 상록수 아래처럼 연중 그늘이 지는 곳의 지피용으로 없어서는 안 될 조경소재이다. 꽃이 피었을 때 모습은 물론 까만 열매도 보기 좋으며 특히 잎이 상록성이라 하층식재용으로 좋다. 꽃말처럼 꽃, 잎, 열매 모두 기쁨을 준다., 물주기: 2주에 1회, 난이도 쉬움, 맥문동은 상록수로 내부 공기를 정화하는 효과가 뛰어나며, 밝은 공간이나 거실, 사무실에 놓아 두면 자연의 느낌을 더해줍니다. 화분은 화이트나 클래식한 디자인의 것을 선택해 심플함을 유지하는 것이 좋습니다., 북유럽, 미니멀리즘, 현대적, 공원, 자연학습장, 주택이나 건물주변의 정원 등에 지면을 덮는 용으로 흔히 쓰인다. 소나무 군식지나 낙엽수 아래 군락으로 모아 심으면 곧 전체에 퍼져 경관을 연출한다. 뿌리를 약용으로 쓰기 위해 집단 재배되기도 한다. 추위와 습해에는 다소 약한 편이지만 서울의 대부분 지역에서 잘 견딘다., 씨앗으로 번식할 때는 가을에 종자를 따 껍질을 벗겨 땅속에 묻어두었다 봄에 뿌리면 싹이 잘 나온다. 땅속의 땅속줄기(지하경)를 뻗으며 자라며 덩이뿌리(괴근)를 달고 있다. 이 덩이뿌리를 몇 개 붙여 포기나누기 하면 쉽게 번식된다., 백합과에 속하는 다년초이다. 우리나라, 중국, 베트남, 일본 등 산성화된 숲에서 5-6종이 자생한다. 우리나라에는 유사종으로 소엽맥문동, 개맥문동이 있다..  강아지에게 안전하다. 고양이에게 안전하다. #845FCD #3C2350 #304A2A #806D94 #A5B391 #3D5419', 'score': -6.717528820037842, 
 'all_ranked': [
    ('기쁨의 연속, 상록수 아래처럼 연중 그늘이 지는 곳의 지피용으로 없어서는 안 될 조경소재이다. 꽃이 피었을 때 모습은 물론 까만 열매도 보기 좋으며 특히 잎이 상록성이라 하층식재용으로 좋다. 꽃말처럼 꽃, 잎, 열매 모두 기쁨을 준다., 물주기: 2주에 1회, 난이도 쉬움, 맥문동은 상록수로 내부 공기를 정화하는 효과가 뛰어나며, 밝은 공간이나 거실, 사무실에 놓아 두면 자연의 느낌을 더해줍니다. 화분은 화이트나 클래식한 디자인의 것을 선택해 심플함을 유지하는 것이 좋습니다., 북유럽, 미니멀리즘, 현대적, 공원, 자연학습장, 주택이나 건물주변의 정원 등에 지면을 덮는 용으로 흔히 쓰인다. 소나무 군식지나 낙엽수 아래 군락으로 모아 심으면 곧 전체에 퍼져 경관을 연출한다. 뿌리를 약용으로 쓰기 위해 집단 재배되기도 한다. 추위와 습해에는 다소 약한 편이지만 서울의 대부분 지역에서 잘 견딘다., 씨앗으로 번식할 때는 가을에 종자를 따 껍질을 벗겨 땅속에 묻어두었다 봄에 뿌리면 싹이 잘 나온다. 땅속의 땅속줄기(지하경)를 뻗으며 자라며 덩이뿌리(괴근)를 달고 있다. 이 덩이뿌리를 몇 개 붙여 포기나누기 하면 쉽게 번식된다., 백합과에 속하는 다년초이다. 우리나라, 중국, 베트남, 일본 등 산성화된 숲에서 5-6종이 자생한다. 우리나라에는 유사종으로 소엽맥문동, 개맥문동이 있다..  강아지에게 안전하다. 고양이에게 안전하다. #845FCD #3C2350 #304A2A #806D94 #A5B391 #3D5419', -6.717528820037842), 
    ('안심하세요, 이름만 들으면 볼품없을 것 같지만 이른 봄 햇빛이 좀 드는 숲 따뜻한 곳에 무리지어 있는 모습을 보면 누구나 감탄할 정도로 아름답다. 키는 작지만 올망졸망 모여서 나고 가늘고 긴 꽃대에 보라색 꽃을 피우며 연잎처럼 생긴 잎을 가지고 있다., 물주기: 1주일에 1회, 난이도 쉬움, 깽깽이풀은 밝은 곳에서 잘 자라며, 화이트 또는 우드톤의 선반이나 테이블 위에 놓아두면 공간을 생동감 있게 연출할 수 있습니다. 또한, 식물의 고유의 느낌을 살리기 위해 같은 스타일의 화분을 이용하여 여러 개를 줄지어 배치하면 더욱 멋진 인테리어를 완성할 수 있습니다., 스칸디나비안, 내추럴, 미니멀리즘, 꽃은 물론 잎 모양도 뛰어나 화단은 물론 분화용으로도 훌륭한 소재다. 특히 추위에 강해 거의 전국에서 월동한다. 자생지에서는 무척 귀하게 여겨져 환경부 지정 보호야생식물 27호이다. 뿌리는 약용으로 쓰이기도 한다., 화단용은 바람이 잘 통하면서 반 그늘진 곳에 심는다. 분화용은 여름 고온기에 약 50%정도 차광이 필요하다. 포기나누기나 씨앗으로 번식하는데 씨앗은 뿌린 후 3년은 지나야 꽃을 볼 수 있다. 씨앗번식과 관련해서 재미난 것은 깽깽이풀 씨앗에 밀선이 있어서 여기에 들어있는 당분을 개미들이 영양원으로 사용한다. 개미는 생존을 위해 당분을 이용하고 깽깽이풀 역시 종피에 당이 벗겨지면서 쉽게 발아하게 되고, 또 개미가 이동한 만큼 자손을 멀리 퍼트릴 수 있게 되어 서로 공생관계에 있다., 매자나무과 여러해살이풀로써 지구상에 딱 2종이 있는데, 그 중 하나가 우리나라에 나는 것이다. 제주와 남부도서지방을 제외한 전국의 숲 가장자리 그늘진 곳에 자생한다..  강아지에게 안전하다. 고양이에게 안전하다. #4D3928 #978A7D #B7A3D6 #3D2321 #A193C1 #403834', -7.5397562980651855), 
    ("인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 물주기: 1주일에 1회, 난이도 쉬움, 칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다., 모던, 미니멀리즘, 보헤미안, 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다., 돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.. 공기정화식물이다. 강아지에게 안전하다. 고양이에게 안전하다. #D43737 #333F25 #DE915A #4A3007 #3B3025 #B47C69", -7.794524192810059), 
    ('길들여지지 않음, 변치 않는 귀여움, 하늘에 흘러가는 구름을 닮고픈 걸까, 이글거리는 태양을 닮고픈 걸까. 여러 자생 나리들 중에서 진짜 나리는 나야! 라고 소리치는 듯 하다. 당당하게 하늘을 우러러 보는 모습이 때 묻지 않은 산골소녀의 해맑은 모습을 보는 것 같기도 하다., 물주기: 주 1회, 난이도 쉬움, 하늘나리는 밝은 색상의 화분에 심어 자연스러운 느낌을 주며, 곁에 다른 식물들과 함께 배치하여 생동감 있는 공간을 연출할 수 있습니다., 모던, 스칸디나비아, 보헤미안, 꽃이 화려한데다 크고 색상이 강렬하여 눈에 잘 뜨인다. 정원의 경계부나 화단의 포인트가 될 부분에 모아심기 해 두면 참 잘 어울린다. 가정에 심어두고 한창 개화기 때 꽃을 잘라 꽃꽂이용으로 이용해도 좋다., 여름철 고온다습한 환경에 약하므로 배수가 잘되는 곳에 심는다. 가장 널리 이용하는 방법이 인편번식으로 깨끗한 삽목상에 심는다. 종자번식도 잘되는데 씨앗을 뿌리면 3년은 지나야 꽃이 핀다. 나리류 중에서 참나리만 주아(꽃대의 잎겨드랑이에 검은색 열매처럼 보이는)가 달리는데 이 주아를 뿌린다., 백합과의 여러해살이 풀이다. 깊은 산 햇빛이 잘 드는 곳에서 자라며 긴 선형의 잎이 어긋나게 달린다. 짙은 주홍빛에 자주색반점이 있는 꽃이 하늘을 향해 핀다..  강아지에게 안전하다. 고양이에게 안전하다. #283B27 #667D3F #474913 #918B42 #6A8136 #E95124', -8.49479866027832), 
    ('하얀 미소, 바닷가나 산자락 아래 평지에서 여름이면 바닥에 하얗게 깔리면서 피는 꽃이다. 찔레랑 형제간 사이라 꽃 모양이 비슷해 보이지만 자세히 보면 많이 다르다., 물주기: 1주일에 1회, 난이도 쉬움, 돌가시나무는 밝은 곳에 두거나 간접광이 드는 자리에 배치하면 좋습니다. 소형 화분에 심어 테이블이나 창가에 두거나, 여러 개의 화분을 군집 배치하여 자연스러운 느낌을 연출할 수 있습니다., 스칸디나비안, 미니멀리즘, 보헤미안, 전남이나 경남의 남부 해안가에 주로 자생한다. 겨울이 춥지 않은 남부지역에서는 정원이나 울타리용으로 이용할 수 있다. 꽃에 향기가 나며 나중에 열매가 빨갛고 둥글게 달린다., 장미과답게 몸 전체에 가시가 많다. 정원이나 자연학습장 같은 생태공원의 바위틈, 생울타리, 경계부분 등에 심으면 잘 어울린다. 햇볕이 충분히 들고 거름기가 좀 있는 곳을 좋아한다., 반상록성의 덩굴성으로 장미과에 속한다. 우리나라, 일본, 중국 남부, 대만 등 아열대 지역에 자생한다..  강아지에게 안전하다. 고양이에게 안전하다. #3E4A1D #9EA170 #A8A082 #5A6031 #B2B297 #586D31', -9.474996566772461)]}